<a href="https://colab.research.google.com/github/Deepanshu-saini/Enzyme-stability-prediction/blob/main/enzyme_stability_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv("train.csv")
train.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [ ]:
train.drop('data_source', axis=1, inplace=True)
train.head()

,seq_id,protein_sequence,pH,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5


In [ ]:
import string
columns = ['ps']
train_df = pd.DataFrame(columns=columns)
for row in train["protein_sequence"]:
    molecule = list(row)
    molecule = pd.Series(data = molecule)
    number_of_el = molecule.value_counts()
    number_of_el_s = number_of_el.to_dict()
    s=0
    for x,y in number_of_el_s.items():
      s+=((ord(x)-65)+1)*y
    s={'ps':s}
    train_df = train_df.append(s, ignore_index=True)
    
train_df.head()

,ps
0,3848
1,3212
2,5750
3,3011
4,19461


In [ ]:
train_df[["seq_id", "pH", "tm"]] = train[["seq_id", "pH", "tm"]]
train_df.head()

,ps,seq_id,pH,tm
0,3848,0,7.0,75.7
1,3212,1,7.0,50.5
2,5750,2,7.0,40.5
3,3011,3,7.0,47.2
4,19461,4,7.0,49.5


In [ ]:
train_df = train_df.fillna(0)
train_df.describe()

,ps,seq_id,pH,tm
count,31390.000000,31390.000000,31390.000000,31390.000000
mean,5435.885282,15694.500000,6.829542,49.147337
std,8020.134221,9061.656811,1.733342,14.010089
min,69.000000,0.000000,0.000000,-1.000000
25%,2367.000000,7847.250000,7.000000,42.100000
50%,4032.000000,15694.500000,7.000000,48.000000
75%,6342.000000,23541.750000,7.000000,53.800000
max,415904.000000,31389.000000,64.900000,130.000000


In [ ]:
list_of_lenths = []
for row in train["protein_sequence"]:
    lenth = len(row)
    list_of_lenths.append(lenth)

train_df["mol_lenth"] = pd.Series(list_of_lenths)
train_df.head()

,ps,seq_id,pH,tm,mol_lenth
0,3848,0,7.0,75.7,341
1,3212,1,7.0,50.5,286
2,5750,2,7.0,40.5,497
3,3011,3,7.0,47.2,265
4,19461,4,7.0,49.5,1451


In [ ]:
X = train_df.drop(["tm", "seq_id"], axis=1)

y = train_df["tm"]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[-0.19799052,  0.09834226, -0.16648415],
       [-0.27729221,  0.09834226, -0.25232525],
       [ 0.03916639,  0.09834226,  0.07699242],
       ...,
       [-0.48689303,  0.09834226, -0.49892332],
       [ 0.2225827 ,  0.09834226,  0.22682416],
       [ 0.20300666,  0.09834226,  0.13942231]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
pred=model.predict(X_test)
pred

array([61.147, 51.124, 49.883, ..., 55.405, 52.501, 62.761])

In [ ]:

from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.metrics import mean_squared_error
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test , pred)
rmse = np.sqrt(mean_squared_error(y_test, pred))
r_squared = r2_score(y_test, pred)
print(mae,mse,rmse,r_squared)

8.87493456884592 155.4538688615883 12.468114086003075 0.18839500370284668


In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
pr=reg.predict(X_test)
pr

array([49.37388105, 49.30998044, 49.0496829 , ..., 49.72104332,
       48.921291  , 49.14065641])

In [ ]:
mae = mean_absolute_error(y_test, pr)
mse = mean_squared_error(y_test , pr)
rmse = np.sqrt(mean_squared_error(y_test, pr))
r_squared = r2_score(y_test, pr)
print(mae,mse,rmse,r_squared)

9.662321601585397 190.52827932256432 13.80319815559294 0.0052759409174033545


In [ ]:
def cost_function(X, y, w, b):
    cost = np.sum((((X.dot(w) + b) - y) ** 2) / (2*len(y)))
    return cost

In [ ]:
def gradient_descent_function(X, y, w, b, alpha=0.01, epochs=1000):
    m = len(y)
    costs = [0] * epochs
    
    for epoch in range(epochs):
        z = X.dot(w) + b
        
        loss = z - y
        
        weight_gradient = X.T.dot(loss) / m
        bias_gradient = np.sum(loss) / m
        
        w = w - alpha*weight_gradient
        b = b - alpha*bias_gradient
        
        cost = cost_function(X, y, w, b)
        costs[epoch] = cost
        
    return w, b, costs

In [ ]:
w = np.random.randn(X_train.shape[1])
b = 0
weights, bias, costs = gradient_descent_function(X_train, y_train, w, b, epochs=2500);

In [ ]:
def predlr(X,w,b):
  return (X.dot(w) + b)

In [ ]:
def r2score(y_pred, y):
    rss = np.sum((y_pred - y) ** 2)
    tss = np.sum((y-y.mean()) ** 2)
    
    r2 = 1 - (rss / tss)
    return r2

In [ ]:
y_p=predlr(X_test,weights,bias)
r2=r2score(y_p,y_test)
r2

0.002290251158728829

In [ ]:
y_p

array([48.98779572, 49.17083799, 49.13825893, ..., 49.20781423,
       49.11789594, 49.44441819])